In [1]:
import tkinter as tk
from tkinter import *
from tkinter import filedialog
from cv2 import imread, imwrite
import numpy as np

In [2]:
def str2bin(string):
    return ''.join((bin(ord(i))[2:]).zfill(7) for i in string)

def bin2str(string):
    return ''.join(chr(int(string[i:i+7], 2)) for i in range(0, len(string), 7))

In [3]:
from PIL import Image

def encode(input_filepath, text, output_filepath):
    try:
        data_length = bin(len(text))[2:].zfill(32)
        bin_data = iter(data_length + str2bin(text))
        img = imread(input_filepath, 1)
        height, width = img.shape[0], img.shape[1]

        for i in range(height):
            for j in range(width):
                pixel = img[i, j]
                for k in range(3):
                    try:
                        x = next(bin_data)
                    except StopIteration:
                        imwrite(output_filepath, img)
                        encoded_img = Image.open(output_filepath)
                        encoded_img.show()
                        return

                    if x == '0' and pixel[k] % 2 == 1:
                        pixel[k] -= 1
                    elif x == '1' and pixel[k] % 2 == 0:
                        pixel[k] += 1

        raise ValueError("Insufficient space in the image to encode the data.")
    except Exception as e:
        raise Exception(f"Error encoding data: {str(e)}")

In [5]:
import cv2

def decode(input_filepath, output_text_filepath):
    try:
        img = cv2.imread(input_filepath)
        height, width = img.shape[0], img.shape[1]

        data_length_binary = ''
        for i in range(32):
            pixel = img[i // width, i % width]
            data_length_binary += str(pixel[0] % 2)

        data_length = int(data_length_binary, 2)
        bin_data = ''

        for i in range(32, 32 + data_length * 7 * 3):
            pixel = img[i // width, i % width]
            bin_data += str(pixel[0] % 2)

        decoded_text = bin2str(bin_data)

        # Save the decoded text to a file
        with open(output_text_filepath, 'w') as output_file:
            output_file.write(decoded_text)

        return decoded_text

    except Exception as e:
        raise Exception(f"Error decoding data: {str(e)}")


In [20]:
import tkinter as tk
from tkinter import ttk, filedialog, messagebox
from cv2 import imread, imwrite
from PIL import Image, ImageTk



def encode_data(image_path, data):
    img = imread(image_path)
    binary_data = ''.join(format(ord(char), '08b') for char in data)
    data_index = 0

    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            for c in range(img.shape[2]):
                if data_index < len(binary_data):
                    img[i, j, c] = int(format(img[i, j, c], '08b')[:-1] + binary_data[data_index], 2)
                    data_index += 1
                else:
                    break
            if data_index == len(binary_data):
                break
    return img

def decode_data(image_path):
    img = imread(image_path)
    binary_data = ""

    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            for c in range(img.shape[2]):
                binary_data += format(img[i, j, c], '08b')[-1]

    byte_array = bytearray()
    for i in range(0, len(binary_data), 8):
        if i + 8 <= len(binary_data):
            byte_array.append(int(binary_data[i:i+8], 2))

    decoded_data = byte_array.decode('utf-8')
    return decoded_data

def browse_image():
    file_path = filedialog.askopenfilename(filetypes=[("Image files", "*.png;*.jpg;*.jpeg")])
    entry_image.delete(0, tk.END)
    entry_image.insert(0, file_path)
    show_image(file_path, label_original)

def encode():
    image_path = entry_image.get()
    data = entry_data.get("1.0", tk.END).strip()

    if image_path and data:
        encoded_img = encode_data(image_path, data)
        imwrite("encoded_image.png", encoded_img)

        show_image("encoded_image.png", label_encoded)
        tk.messagebox.showinfo("Encoding", "Data has been successfully encoded.")
    else:
        tk.messagebox.showerror("Error", "Please select an image and enter data.")
def decode():
    image_path = entry_image.get()

    if image_path:
        decoded_data = decode_data(image_path)
        entry_data.delete("1.0", tk.END)
        entry_data.insert(tk.END, decoded_data)

        # Show the decoded message in a messagebox
        tk.messagebox.showinfo("Decoding", f"Decoded Message:\n{decoded_data}")
    else:
        tk.messagebox.showerror("Error", "Please select an image.")
        
def decode_data(image_path):
    img = imread(image_path)
    binary_data = ""

    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            for c in range(img.shape[2]):
                binary_data += format(img[i, j, c], '08b')[-1]

    decoded_data = ""
    for i in range(0, len(binary_data), 8):
        if i + 8 <= len(binary_data):
            decoded_data += chr(int(binary_data[i:i+8], 2))
    return decoded_data


def show_image(image_path, target_label):
    img = Image.open(image_path)
    img = img.resize((300, 300), Image.ANTIALIAS if hasattr(Image, 'ANTIALIAS') else Image.BICUBIC)
    img = ImageTk.PhotoImage(img)
    target_label.config(image=img)
    target_label.image = img

# GUI setup
root = tk.Tk()
root.title("Steganography Tool")

frame = ttk.Frame(root, padding="10")
frame.grid(column=0, row=0, sticky=(tk.W, tk.E, tk.N, tk.S))

# Entry widget for image path
label_image = ttk.Label(frame, text="Select Image:")
label_image.grid(column=0, row=0, sticky=tk.W)

entry_image = ttk.Entry(frame, width=40)
entry_image.grid(column=1, row=0, sticky=(tk.W, tk.E), padx=10, pady=5)

button_browse = ttk.Button(frame, text="Browse", command=browse_image)
button_browse.grid(column=2, row=0, sticky=tk.W, pady=5)

# Original image display
label_original = ttk.Label(frame, text="Original Image")
label_original.grid(column=0, row=3, columnspan=3, pady=10)

# Text widget for data input
label_data = ttk.Label(frame, text="Enter Data:")
label_data.grid(column=0, row=1, sticky=tk.W)

entry_data = tk.Text(frame, width=40, height=5)
entry_data.grid(column=1, row=1, sticky=(tk.W, tk.E), padx=10, pady=5)

# Buttons for encoding and decoding
button_encode = ttk.Button(frame, text="Encode", command=encode)
button_encode.grid(column=0, row=2, sticky=tk.W, pady=10)

button_decode = ttk.Button(frame, text="Decode", command=decode)
button_decode.grid(column=1, row=2, sticky=tk.W, pady=10)

# Encoded image display
label_encoded = ttk.Label(frame, text="Encoded Image")
label_encoded.grid(column=0, row=4, columnspan=3, pady=10)

# Label for decoded message
label_decoded = ttk.Label(frame, text="Decoded Message:")
label_decoded.grid(column=0, row=5, columnspan=3, pady=10)

root.mainloop()


In [13]:
import tkinter as tk
from tkinter import ttk, filedialog, messagebox
from cv2 import imread, imwrite
from PIL import Image, ImageTk

def encode_data(image_path, data):
    img = imread(image_path)
    binary_data = ''.join(format(ord(char), '08b') for char in data)
    data_index = 0

    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            for c in range(img.shape[2]):
                if data_index < len(binary_data):
                    img[i, j, c] = int(format(img[i, j, c], '08b')[:-1] + binary_data[data_index], 2)
                    data_index += 1
                else:
                    break
            if data_index == len(binary_data):
                break
    return img

def process_image(image_path, operation, data=None):
    img = imread(image_path)
    result_data = ""

    if operation == "encode" and data:
        binary_data = ''.join(format(ord(char), '08b') for char in data)
        data_index = 0

        for i in range(img.shape[0]):
            for j in range(img.shape[1]):
                for c in range(img.shape[2]):
                    if data_index < len(binary_data):
                        img[i, j, c] = int(format(img[i, j, c], '08b')[:-1] + binary_data[data_index], 2)
                        data_index += 1
                    else:
                        break
                if data_index == len(binary_data):
                    break

    elif operation == "decode":
        binary_data = "".join(format(img[i, j, c], '08b')[-1] for i in range(img.shape[0]) for j in range(img.shape[1]) for c in range(img.shape[2]))
        result_data = "".join(chr(int(binary_data[i:i+8], 2)) for i in range(0, len(binary_data), 8) if i + 8 <= len(binary_data))

    return img, result_data

def browse_image():
    file_path = filedialog.askopenfilename(filetypes=[("Image files", "*.png;*.jpg;*.jpeg")])
    entry_image.delete(0, tk.END)
    entry_image.insert(0, file_path)
    show_image(file_path, label_original)

def process_and_show(operation):
    image_path = entry_image.get()

    if image_path:
        data = entry_data.get("1.0", tk.END).strip()

        result_img, result_data = process_image(image_path, operation, data)

        if operation == "encode":
            imwrite("encoded_image.png", result_img)
            show_image("encoded_image.png", label_encoded)
            tk.messagebox.showinfo("Encoding", "Data has been successfully encoded.")
        elif operation == "decode":
            entry_data.delete("1.0", tk.END)
            entry_data.insert(tk.END, result_data)
            tk.messagebox.showinfo("Decoding", f"Decoded Message:\n{result_data}")

    else:
        tk.messagebox.showerror("Error", "Please select an image.")

def show_image(image_path, target_label):
    img = Image.open(image_path)
    img = img.resize((300, 300), Image.ANTIALIAS if hasattr(Image, 'ANTIALIAS') else Image.BICUBIC)
    img = ImageTk.PhotoImage(img)
    target_label.config(image=img)
    target_label.image = img

# GUI setup
root = tk.Tk()
root.title("Steganography Tool")

frame = ttk.Frame(root, padding="10")
frame.grid(column=0, row=0, sticky=(tk.W, tk.E, tk.N, tk.S))

# Entry widget for image path
label_image = ttk.Label(frame, text="Select Image:")
label_image.grid(column=0, row=0, sticky=tk.W)

entry_image = ttk.Entry(frame, width=40)
entry_image.grid(column=1, row=0, sticky=(tk.W, tk.E), padx=10, pady=5)

button_browse = ttk.Button(frame, text="Browse", command=browse_image)
button_browse.grid(column=2, row=0, sticky=tk.W, pady=5)

# Original image display
label_original = ttk.Label(frame, text="Original Image")
label_original.grid(column=0, row=3, columnspan=3, pady=10)

# Text widget for data input
label_data = ttk.Label(frame, text="Enter Data:")
label_data.grid(column=0, row=1, sticky=tk.W)

entry_data = tk.Text(frame, width=40, height=5)
entry_data.grid(column=1, row=1, sticky=(tk.W, tk.E), padx=10, pady=5)

# Buttons for encoding and decoding
button_encode = ttk.Button(frame, text="Encode", command=lambda: process_and_show("encode"))
button_encode.grid(column=0, row=2, sticky=tk.W, pady=10)

button_decode = ttk.Button(frame, text="Decode", command=lambda: process_and_show("decode"))
button_decode.grid(column=1, row=2, sticky=tk.W, pady=10)

# Encoded image display
label_encoded = ttk.Label(frame, text="Encoded Image")
label_encoded.grid(column=0, row=4, columnspan=3, pady=10)

# Label for decoded message
label_decoded = ttk.Label(frame, text="Decoded Message:")
label_decoded.grid(column=0, row=5, columnspan=3, pady=10)

root.mainloop()


In [12]:
import tkinter as tk
from tkinter import ttk, filedialog, messagebox
import os
import subprocess

def browse_files(label):
    filename = filedialog.askopenfilename(initialdir="/", title="Select a File", filetypes=(("Text files", "*.txt"),))
    label.configure(text=filename)

def browse_image(label):
    filename = filedialog.askopenfilename(initialdir="/", title="Select an Image", filetypes=(("Images", "*.png;*.jpg;*.jpeg"),))
    label.configure(text=filename)

def display_message(message):
    messagebox.showinfo("Information", message)

def open_hidden_image(output_path):
    if os.path.exists(output_path):
        subprocess.Popen(["xdg-open", output_path])  # Open with default image viewer
    else:
        display_message("Hidden image not found. Please hide data first.")

def hide_data():
    if label_file_explorer1.cget("text") == "data file" or label_file_explorer2.cget("text") == "image file" or label_file_explorer1.cget("text") == "" or label_file_explorer2.cget("text") == "":
        display_message("You didn't select both data and image files")
    else:
        data_file_path = label_file_explorer1.cget("text")
        image_file_path = label_file_explorer2.cget("text")
        output_directory = filedialog.askdirectory()

        with open(data_file_path, 'rb') as data_file:
            data = data_file.read()

        with open(image_file_path, 'rb') as image_file:
            image_data = bytearray(image_file.read())

        # Embed data into image_data
        for i in range(len(data)):
            image_data[i] = (image_data[i] & 0xFE) | ((data[i] >> 7) & 0x01)
            if i + 1 < len(image_data):
                image_data[i + 1] = (image_data[i + 1] & 0x7F) | ((data[i] & 0x7F) << 1)

        output_path = os.path.join(output_directory, "hidden_image.png")

        with open(output_path, 'wb') as output_file:
            output_file.write(image_data)

        display_message(f"Data hidden in the image.\nSaved as '{output_path}'")

def open_hidden_image_wrapper():
    output_path = os.path.join(os.getcwd(), "hidden_image.png")
    open_hidden_image(output_path)

# Create the root window
root = tk.Tk()
root.title('Data Hiding')

# Create a style to configure ttk widgets
style = ttk.Style()
style.configure('TButton', padding=(10, 5), font=('Helvetica', 12))
style.configure('TLabel', font=('Helvetica', 12))
style.configure('TFrame', padding=(20, 10))

# Create a main frame
main_frame = ttk.Frame(root, padding=(20, 10))
main_frame.grid(row=0, column=0, sticky=(tk.W, tk.E, tk.N, tk.S))

# Create labels
label_file_explorer1 = ttk.Label(main_frame, text="data file")
label_file_explorer2 = ttk.Label(main_frame, text="image file")

# Create buttons
button_explore1 = ttk.Button(main_frame, text="Choose data file", command=lambda: browse_files(label_file_explorer1))
button_explore2 = ttk.Button(main_frame, text="Choose image file", command=lambda: browse_image(label_file_explorer2))
button_hide = ttk.Button(main_frame, text="Hide data in image", command=hide_data)
button_open_hidden_image = ttk.Button(main_frame, text="Open Hidden Image", command=open_hidden_image_wrapper)

# Grid layout
label_file_explorer1.grid(row=0, column=0, pady=10)
button_explore1.grid(row=0, column=1, pady=10)
label_file_explorer2.grid(row=1, column=0, pady=10)
button_explore2.grid(row=1, column=1, pady=10)
button_hide.grid(row=2, column=0, columnspan=2, pady=20)
button_open_hidden_image.grid(row=3, column=0, columnspan=2, pady=10)

# Run the Tkinter main loop
root.mainloop()


In [21]:
import cv2
import numpy as np

In [47]:
import cv2
import numpy as np

def to_bin(data):
    """Convert `data` to binary format as string"""
    if isinstance(data, str):
        return ''.join([ format(ord(i), "08b") for i in data ])
    elif isinstance(data, bytes):
        return ''.join([ format(i, "08b") for i in data ])
    elif isinstance(data, np.ndarray):
        return [ format(i, "08b") for i in data ]
    elif isinstance(data, int) or isinstance(data, np.uint8):
        return format(data, "08b")
    else:
        raise TypeError("Type not supported.")
    
def encode(image_name, secret_data):
    # read the image
    image = cv2.imread(image_name)
    # maximum bytes to encode
    n_bytes = image.shape[0] * image.shape[1] * 3 // 8
    print("[*] Maximum bytes to encode:", n_bytes)
    if len(secret_data) > n_bytes:
        raise ValueError("[!] Insufficient bytes, need bigger image or less data.")
    print("[*] Encoding data...")
    # add stopping criteria
    secret_data += "====="
    data_index = 0
    # convert data to binary
    binary_secret_data = to_bin(secret_data)
    # size of data to hide
    data_len = len(binary_secret_data)
    for row in image:
        for pixel in row:
            # convert RGB values to binary format
            r, g, b = to_bin(pixel)
            # modify the least significant bit only if there is still data to store
            if data_index < data_len:
                # least significant red pixel bit
                pixel[0] = int(r[:-1] + binary_secret_data[data_index], 2)
                data_index += 1
            if data_index < data_len:
                # least significant green pixel bit
                pixel[1] = int(g[:-1] + binary_secret_data[data_index], 2)
                data_index += 1
            if data_index < data_len:
                # least significant blue pixel bit
                pixel[2] = int(b[:-1] + binary_secret_data[data_index], 2)
                data_index += 1
            # if data is encoded, just break out of the loop
            if data_index >= data_len:
                break
    return image
def decode(image_name):
    print("[+] Decoding...")
    # read the image
    image = cv2.imread(image_name)
    binary_data = ""
    for row in image:
        for pixel in row:
            r, g, b = to_bin(pixel)
            binary_data += r[-1]
            binary_data += g[-1]
            binary_data += b[-1]
    # split by 8-bits
    all_bytes = [ binary_data[i: i+8] for i in range(0, len(binary_data), 8) ]
    # convert from bits to characters
    decoded_data = ""
    for byte in all_bytes:
        decoded_data += chr(int(byte, 2))
        if decoded_data[-5:] == "=====":
            break
    return decoded_data[:-5]  

if __name__ == "__main__":
    input_image = "img/1920RepublicanPresidentialNomination9thBallot.png"
    output_image = "encoded_image.PNG"
    secret_data = "This is a top secret message."
    # encode the data into the image
    encoded_image = encode(image_name=input_image, secret_data=secret_data)
    # save the output image (encoded image)
    cv2.imwrite(output_image, encoded_image)
    # decode the secret data from the image
    decoded_data = decode(output_image)
    print("[+] Decoded data:", decoded_data) 
 

[*] Maximum bytes to encode: 59350
[*] Encoding data...
[+] Decoding...
[+] Decoded data: This is a top secret message.


In [51]:
import cv2
import numpy as np
import tkinter as tk
from tkinter import ttk, filedialog

def to_bin(data):
    if isinstance(data, str):
        return ''.join([format(ord(i), "08b") for i in data])
    elif isinstance(data, bytes):
        return ''.join([format(i, "08b") for i in data])
    elif isinstance(data, np.ndarray):
        return [format(i, "08b") for i in data]
    elif isinstance(data, int) or isinstance(data, np.uint8):
        return format(data, "08b")
    else:
        raise TypeError("Type not supported.")

def encode(image_name, secret_data):
    image = cv2.imread(image_name)
    n_bytes = image.shape[0] * image.shape[1] * 3 // 8
    if len(secret_data) > n_bytes:
        raise ValueError("Insufficient bytes, need bigger image or less data.")
    secret_data += "====="
    data_index = 0
    binary_secret_data = to_bin(secret_data)
    data_len = len(binary_secret_data)
    for row in image:
        for pixel in row:
            r, g, b = to_bin(pixel)
            if data_index < data_len:
                pixel[0] = int(r[:-1] + binary_secret_data[data_index], 2)
                data_index += 1
            if data_index < data_len:
                pixel[1] = int(g[:-1] + binary_secret_data[data_index], 2)
                data_index += 1
            if data_index < data_len:
                pixel[2] = int(b[:-1] + binary_secret_data[data_index], 2)
                data_index += 1
            if data_index >= data_len:
                break
    return image

def decode(image_name):
    image = cv2.imread(image_name)
    binary_data = ""
    for row in image:
        for pixel in row:
            r, g, b = to_bin(pixel)
            binary_data += r[-1]
            binary_data += g[-1]
            binary_data += b[-1]
    all_bytes = [binary_data[i: i + 8] for i in range(0, len(binary_data), 8)]
    decoded_data = ""
    for byte in all_bytes:
        decoded_data += chr(int(byte, 2))
        if decoded_data[-5:] == "=====":
            break
    return decoded_data[:-5]

def browse_image(entry_widget):
    file_path = filedialog.askopenfilename(filetypes=[("Image files", "*.png;*.jpg;*.jpeg")])
    entry_widget.delete(0, tk.END)
    entry_widget.insert(0, file_path)

def encode_data():
    input_image = entry_input_image.get()
    output_image = entry_output_image.get()
    secret_data = entry_secret_data.get("1.0", tk.END).strip()

    if input_image and output_image and secret_data:
        encoded_image = encode(image_name=input_image, secret_data=secret_data)
        cv2.imwrite(output_image, encoded_image)
        tk.messagebox.showinfo("Encoding", "Data has been successfully encoded.")
    else:
        tk.messagebox.showerror("Error", "Please fill in all fields.")

def decode_data():
    input_image = entry_input_image.get()

    if input_image:
        decoded_data = decode(image_name=input_image)
        entry_secret_data.delete("1.0", tk.END)
        entry_secret_data.insert(tk.END, decoded_data)
        tk.messagebox.showinfo("Decoding", f"Data has been successfully decoded: {decoded_data}")
    else:
        tk.messagebox.showerror("Error", "Please fill in all fields.")

# GUI setup
root = tk.Tk()
root.title("Image Steganography Tool")

frame = ttk.Frame(root, padding="10")
frame.grid(column=0, row=0, sticky=(tk.W, tk.E, tk.N, tk.S))

# Entry widget for input image
label_input_image = ttk.Label(frame, text="Input Image:")
label_input_image.grid(column=0, row=0, sticky=tk.W)

entry_input_image = ttk.Entry(frame, width=40)
entry_input_image.grid(column=1, row=0, sticky=(tk.W, tk.E), padx=10, pady=5)

button_browse_input = ttk.Button(frame, text="Browse", command=lambda: browse_image(entry_input_image))
button_browse_input.grid(column=2, row=0, sticky=tk.W, pady=5)

# Entry widget for output image
label_output_image = ttk.Label(frame, text="Output Image:")
label_output_image.grid(column=0, row=1, sticky=tk.W)

entry_output_image = ttk.Entry(frame, width=40)
entry_output_image.grid(column=1, row=1, sticky=(tk.W, tk.E), padx=10, pady=5)

button_browse_output = ttk.Button(frame, text="Browse", command=lambda: browse_image(entry_output_image))
button_browse_output.grid(column=2, row=1, sticky=tk.W, pady=5)

# Text widget for secret data input
label_secret_data = ttk.Label(frame, text="Enter Secret Data:")
label_secret_data.grid(column=0, row=2, sticky=tk.W)

entry_secret_data = tk.Text(frame, width=40, height=5)
entry_secret_data.grid(column=1, row=2, sticky=(tk.W, tk.E), padx=10, pady=5)

# Buttons for encoding and decoding
button_encode = ttk.Button(frame, text="Encode", command=encode_data)
button_encode.grid(column=0, row=3, sticky=tk.W, pady=10)

button_decode = ttk.Button(frame, text="Decode", command=decode_data)
button_decode.grid(column=1, row=3, sticky=tk.W, pady=10)

root.mainloop()
